# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [6]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """,
)

Let's execute the Agent:

In [7]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Overall, yes—bananas are healthy in moderation.
    
    Key benefits:
    - Good source of potassium, vitamin B6, vitamin C, and dietary fiber.
    - Low in fat; convenient, inexpensive, portable.
    
    Considerations:
    - They contain natural sugars and carbs, so portion size matters for blood sugar control (about 27 g carbs for a medium banana).
    - Ripe bananas are sweeter; unripe are firmer with more resistant starch.
    
    Typical serving: 1 medium banana (~105 calories, 27 g carbs, 3 g fiber).
    
    Bottom line: include them as part of a balanced diet; adjust portions if you’re watching sugar or carbs.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [10]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are generally healthy and part of a balanced diet.

Key positives:
- Good source of potassium, vitamin C, vitamin B6, and dietary fiber.
- Contains resistant starch when unripe (benefits gut health).
- Low in fat and cholesterol-free.

Things to consider:
- Moderate amount of natural sugars; portion control matters for diabetes or blood sugar concerns.
- One medium banana ≈ 105 calories.

Bottom line: for most people, 1–2 medium bananas daily are a healthy, convenient fruit option as part of a varied diet.

_Good Job!_